In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services.

In [2]:
import json

from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Import modules

In langrila, the module to chat with LLM and the module to call tools are completely separated. FunctionalChat class is the combination of those two.

- `XXXChatModule`: Only focuses on doing conversation with LLM
- `XXXFunctionCallingModule`: Only focuses on calling tools
- `XXXFunctionalChat`: The combination of the two. FunctionCallingModule works at first and then ChatModule performs. If any tool is not provided, this module behaves as just ChatModule.

In [4]:
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Structured output

OpenAI Chat Completion and Gemini support json mode generation.

### For OpenAI Chat Completion

In [5]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="openai",
    json_mode=True,
)

In [6]:
prompt_template = """# INSTRUCTION
Extract fruit names from the following text and output with the following JSON format. Don't generate JSON code block.

# TEXT
{prompt}

# OUTPUT JSON FORMAT
{{
    "fruits": ["xxx", "xxx", "xxx"]
}}
"""

In [7]:
prompt = prompt_template.format(prompt="This store sells apples, oranges, and bananas.")
response = chat_openai.run(prompt=prompt)
json.loads(response.message.content[0].text)

{'fruits': ['apples', 'oranges', 'bananas']}

Pydantic schema is also available for latest models.

In [8]:
from pydantic import BaseModel


class Step(BaseModel):
    explanation: str
    output: str


class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

In [9]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
    api_type="openai",
    json_mode=True,  # also needed
    response_schema=MathReasoning,
    system_instruction="You are a helpful math tutor. Guide the user through the solution step by step.",
)

In [10]:
prompt = "how can I solve 8x + 7 = -23"
response = chat_openai.run(prompt=prompt)

# resposne message is in JSON string that can be converted to specified schema
resposne_json = json.loads(response.message.content[0].text)
MathReasoning(**resposne_json)

MathReasoning(steps=[Step(explanation='The first step to solving this equation is to isolate the variable term (8x) on one side of the equation. To do this, we need to get rid of the constant term (+7) on the left side. We can subtract 7 from both sides of the equation to achieve this.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Subtracting 7 from both sides results in the cancellation of the +7 on the left side, leaving us only with the variable term on the left.', output='8x = -30'), Step(explanation='Now that we have 8x on the left side, we need to solve for x by getting x by itself. Since x is currently multiplied by 8, we can divide both sides by 8 to isolate x.', output='8x / 8 = -30 / 8'), Step(explanation='Dividing both sides by 8 simplifies the equation. The variable x is now isolated on the left side, and we perform the division on the right side.', output='x = -30 / 8'), Step(explanation='-30 divided by 8 simplifies to -3.75. Thus, the value of x is -3.75.', output='

### For Gemini

In [11]:
from typing_extensions import TypedDict


class ResponseSchema(TypedDict):
    fruits: list[str]

In [12]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-flash",
    json_mode=True,
    response_schema=ResponseSchema,
)

In [13]:
prompt = "This store sells apples, oranges, and bananas."

response = gemini.run(prompt=prompt)

In [14]:
json.loads(response.message.content[0].text)

{'fruits': ['apples', 'oranges', 'bananas']}